In [ ]:
from mastsel import *
mItCPU = Integrator(np, np.float64, '')
mItGPU = Integrator(cp, cp.float64, '')

In [ ]:
NGS_flux = [10000, 30000, 5000]
NGS_SR_1650 = [0.4, 0.2, 0.6]
NGS_FWHM_mas = [90, 110, 85]
mLO = MavisLO("../data/ini", 'mavisParamsTests', 'data/windpsd_mavis.fits')
print(mLO.computeBias(NGS_flux[0], NGS_SR_1650[0], NGS_FWHM_mas[0]))
print(mLO.computeBias(NGS_flux[1], NGS_SR_1650[1], NGS_FWHM_mas[1]))
print(mLO.computeBias(NGS_flux[2], NGS_SR_1650[2], NGS_FWHM_mas[2]))
# si usa nel C1 e C3

In [ ]:
#hdul = fits.open('data/example_gaussian_psf.fits')
hdul = fits.open('../data/mu_kthr.fits')
psf_data1 = np.asarray(hdul[0].data, np.float64)
print(psf_data1.shape)
hdul = fits.open('../data/example_gaussian_psf.fits')
psf_data = np.asarray(hdul[0].data, np.float64)
print(psf_data.shape)

In [ ]:
rh1 = mf['TruncatedMeanBasic'].rhs
dd = {'i_max':30 , 'b':1.0, 't':0.5, 'nu':2.0, 'sigma_RON':mLO.sigmaRON_LO, 'f_k': 5.0}
sexpr1 = subsParamsByName( rh1, dd )
mu_k_value = sexpr1.doit().evalf(5)
print(mu_k_value)

rh1 = mf['TruncatedVarianceBasic'].rhs
dd = {'i_max':30 , 'b':1.0, 't':0.5, 'nu':2.0, 'sigma_RON':mLO.sigmaRON_LO, 'f_k': 5.0, "mu_k_thr":mu_k_value}
sexpr1 = subsParamsByName( rh1, dd )
sigma_k_value = sexpr1.doit().evalf(5)
print(sigma_k_value)

In [ ]:
gridSize = 10
gridSpanArcsec = 10.0
gridSpanRad=gridSpanArcsec/radiansToArcsecs
xVarPixels = yVarPixels = 2.0
pixelSize = gridSpanRad/(gridSize-1)
imax = 32
zmin = 0.0001
zmax = 32.0
integrationPoints = 1024

mIt = Integrator(cp, cp.float64, '')

def sampleCompute(aFunction, expr0, xx):
    nn = xx.size
    aIntegral = sp.Integral(aFunction, (getSymbolByName(aFunction, 'z'), zmin, zmax), (getSymbolByName(aFunction, 'i'), 1.0, int(imax)) )
    paramAndRange = ( 'f_k', 1.0, 10.0, nn, 'linear' )
    lh = sp.Function('B')(getSymbolByName(aIntegral, 'f_k'))
    xplot1, zplot1 = mIt.IntegralEval(lh, aIntegral, [paramAndRange], [ (integrationPoints, 'linear'), (imax, 'linear')], 'raw')
    xplot1, zplot1 = cp.asnumpy(xplot1), cp.asnumpy(zplot1)
    ss, s0 = evaluateFormula(expr0, {}, ['f_k'], [xx] )
    zplot1 = zplot1 + cp.asnumpy(s0)
    return xplot1, zplot1

xplot1, mu_ktr_array = sampleCompute(mLO.aFunctionM, mLO.expr0M, cp.asarray(cp.linspace(1.0, 10, gridSize), dtype=cp.float64))
xplot2, var_ktr_array = sampleCompute(mLO.aFunctionV, mLO.expr0V, cp.asarray(cp.linspace(1.0, 10, gridSize), dtype=cp.float64))
var_ktr_array = var_ktr_array - mu_ktr_array**2
sigma_ktr_array = np.sqrt(var_ktr_array.astype(cp.float64))
fig, ax = plt.subplots(figsize=(10,5))
plt.plot(xplot1[0], mu_ktr_array)
plt.plot(xplot2[0], sigma_ktr_array)
plt.show()

In [ ]:
expr0 = mf['truncatedMeanComponents0']
exprK = mf['truncatedMeanComponents1']
integral = mf['truncatedMeanComponents2']

dd0 = {'t':mLO.ThresholdWCoG_LO, 'nu':mLO.NewValueThrPix_LO, 'sigma_RON':mLO.sigmaRON_LO}
dd1 = {'b':mLO.Dark_LO/mLO.SensorFrameRate_LO}
dd2 = {'F':mLO.ExcessNoiseFactor_LO}
expr0 = subsParamsByName( expr0, {**dd0, **dd1} )
exprK = subsParamsByName( exprK, dd1 )
integral = subsParamsByName( integral,  {**dd0, **dd1, **dd2} )
expr0 = subsParamsByName( expr0, {'f_k':1.0} )
exprK = subsParamsByName( exprK, {'f_k':1.0} )

s0 = expr0.evalf()
print(s0)

for ivalue in range(1,20+1,1):
    exprKi = subsParamsByName(exprK, {'i':ivalue})
    integralI = subsParamsByName(integral, {'i':ivalue, 'z_max':30.0})
    s1 = exprKi.evalf(10) * integralI.evalf(10)
    s0 += s1
    print(ivalue, s0)

In [ ]:
var_k = sp.symbols('sigma^2_k\,thr')
mu_k = sp.symbols('mu_k\,thr')

x = sp.symbols('x', real=True)
i = sp.symbols('i', integer=True, positive=True)
# 8. Fattore di excess noise (dato) (F) : 1.0 (formule semplificate , per F>1 formule complete)
F = sp.symbols('F', real=True)
# f_k = I_k + back, vedi appedice D : back: 0.0
f_k = sp.symbols('f_k', real=True)
# 7. Dark Current (uguale per ogni pixel, dato) (b) : 0.0
b = sp.symbols('b', real=True)
# 9. fattore Threshold (dato) (t) : 0.0
t = sp.symbols('t', real=True)
# 10. valore associato al sotto threshold (dato) (nu) : 0.0
nu = sp.symbols('nu', real=True)
# 5. RON per pixel (uguale per ogni pixel, dato) (sigma_RON) : 0.5, var: 0.25
sigma_ron = sp.symbols('sigma_RON', real=True)

phi = sp.Function('phi')(x)
Phi = sp.Function('Phi')(x)
K_i = sp.Function('K')(i)

with sp.evaluate(False):
    expr_phi = (sp.S(1)/sp.sqrt(sp.S(2)*sp.pi)) * sp.exp( - x**2 / 2) 
    expr_Phi = (sp.S(1)/sp.S(2)) * (1+sp.erf(x/sp.sqrt(sp.S(2)))) 
    expr_K_i = sp.exp(-(f_k+b)) * (f_k+b) ** i / sp.factorial(i)

# Visualization of the formulas 
display( sp.Eq( phi, expr_phi ) )
display( sp.Eq( Phi, expr_Phi ) )
display( sp.Eq( K_i, expr_K_i ) )
f1 = phi.subs({x: (t-(i-b))/sigma_ron})
f2 = Phi.subs({x: (i-b-t)/sigma_ron})
f3 = Phi.subs({x: (t-(i-b))/sigma_ron})
fK = K_i
i_max = sp.oo
sexpr1 = sp.Sum(fK * ( sigma_ron * f1  + (i-b) * f2 + nu * f3 ) , (i, 0, i_max))
sexpr2 = sp.Sum(fK * ( sigma_ron * (t+i-b) * f1 + (sigma_ron**2 + (i-b)**2) * f2 + nu**2 * f3 ) , (i, 0, i_max)) - mu_k**2
display(sp.Eq(mu_k,sexpr1))
display(sp.Eq(var_k,sexpr2))

z = sp.symbols('z', real=True, positive=True)
G = sp.Function('G')(z, i)

expr_G = z ** (i/(F-1) -1 )  * sp.exp(-z/(F-1)) / ( (F-1)**(i/(F-1)) *  sp.gamma(i/(F-1)) )
display( sp.Eq( G, expr_G ) )

f1 = phi.subs({x: (t+b)/sigma_ron})
f2 = Phi.subs({x: -(t+b)/sigma_ron})
f3 = Phi.subs({x: (t+b)/sigma_ron})
f4 = phi.subs({x: (t-(z-b))/sigma_ron})
f5 = Phi.subs({x: (z-b-t)/sigma_ron})
f6 = Phi.subs({x: (t-(z-b))/sigma_ron})
GG = G
fK = K_i
i_max = sp.oo
expr10 = sigma_ron * f1  - b * f2 + nu * f3
integrand_1 = GG * ( sigma_ron * f4 + (z-b) * f5 + nu * f6 )
expr1 =  sp.exp(-(f_k+b)) * expr10 + sp.Sum(fK * sp.Integral( integrand_1, (z, 0, sp.oo)) , (i, 1, i_max))
expr20 = sigma_ron * (t-b) * f1 + (sigma_ron**2 + b**2) * f2 + nu**2 * f3
integrand_2 = GG * (sigma_ron * (t+z-b) * f4 + (sigma_ron**2 + (z-b)**2) * f5 + nu**2 * f6)
expr2 = sp.exp(-(f_k+b)) * expr20 + sp.Sum(fK * sp.Integral( integrand_2, (z, 0, sp.oo)) , (i, 1, i_max)) - mu_k**2
display(sp.Eq(mu_k,expr1))
display(sp.Eq(var_k,expr2))

In [ ]:
# Inputs: 
# 1. vettore di posizioni (x,y) dei pixel x_k
# 2. vettore di pesi scalari dei pixel w_k (per ora solo 0 oppure 1...)
# 3. vettore di flussi I_k su ogni pixel per lo spot
# 4. vettore di flussi I_k' su ogni pixel per lo spot shiftato di p_offset (i.e. 1/4) di pixel
# nota I_k viene calcolato considerando un modello gaussiano 2d di FWHM=n pixel (ad esenpio 2) (ricavi varianza)
# 5. RON per pixel (uguale per ogni pixel, dato) (sigma_RON) : 0.0
# 6. Valore di background (uguale per ogni pixel, dato) (back) : 0.0
# 7. Dark Current (uguale per ogni pixel, dato) (b) : 0.0
# 8. Fattore di excess noise (dato) (F) : 1.0
# 9. valore Threshold (dato) (t) : 0.0
# 10. valore associato a sotto*threshold (dato) (nu) : 0.0
# Calcolo intermedio grosso
# Calcolo mu_ktr, sigma_ktr, formulone in appendice D, dove:
# f_k = I_k + back, vedi appedice D
# Output:
# Due numeri:
# 1. Varianza della slope (si calcola solo per x e si assume uguale per y) (prima formula pag 26)
#    dove mu e sigma sono calcolati con I_k
# Ripeto il calcolo della mu' con I_k' per calcolare il bias
# 2. Bias Factor = mu' / p_offset

#    dd0 = {'t':0.5, 'nu':2.0, 'sigma_RON':sigmaRON}
#    dd1 = {'b':1.0}
